# CREDIT REPORT - FEATURE ENGINEERING

This notebooks contains the analysis for the first assignment. It includes the following steps:

1. Loading data
2. Data analysis by data category:
    - Understanding data
    - Proposal of features
    - Feature engineering function.
3. Summary feature engineering class


---

## Loading Data

Loading JSON from URL and Flatten data into a pandas DataFrame

We'll flatten the data on level 2 to treat each subcategories as a separate DataFrame for clarity. 

---

In [1]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen, urlretrieve
import datetime
from dateutil import relativedelta

In [3]:
with open('credit_report.json') as f:
    data = json.load(f)
    
df = pd.json_normalize(data, max_level=2, sep='_', record_prefix = False)
df.shape

In [5]:
df.columns

Index(['application_id', 'data_consumerfullcredit_subjectlist',
       'data_consumerfullcredit_accountrating',
       'data_consumerfullcredit_enquirydetails',
       'data_consumerfullcredit_guarantorcount',
       'data_consumerfullcredit_guarantordetails',
       'data_consumerfullcredit_telephonehistory',
       'data_consumerfullcredit_employmenthistory',
       'data_consumerfullcredit_enquiryhistorytop',
       'data_consumerfullcredit_creditaccountsummary',
       'data_consumerfullcredit_deliquencyinformation',
       'data_consumerfullcredit_creditagreementsummary',
       'data_consumerfullcredit_personaldetailssummary',
       'data_consumerfullcredit_accountmonthlypaymenthistory',
       'data_consumerfullcredit_accountmonthlypaymenthistoryheader',
       'data_consumerfullcredit_identificationhistory'],
      dtype='object')

There are 16 sub-categories of data in the credit report. Let's check them individually.

----

## Data Analysis

We will analyze data separately for each category. For each category we will :
- Check the data
- Get an intuition on the features extract
- Define a function for feature extraction


> Note: I did not look at 'credit.csv' file before doing this exercise so I did not try to replicate any fields from that file here.

---

In [6]:
TODAY = datetime.datetime.today()

---

### Account Ratings

In [7]:
account_rating = pd.DataFrame(list(df['data_consumerfullcredit_accountrating'])).astype(int)
account_rating.columns = account_rating.columns.str.replace('noof', '').str.replace('accounts', '_').str.replace('ccounts', '_')
account_rating.head()

,other_bad,other_good,retail_bad,retail_good,telecom_bad,autoloan_bad,autoloan_good,homeloan_bad,telecom_good,homeloan_good,jointloan_bad,studyloan_bad,creditcard_bad,jointloan_good,studyloan_good,creditcard_good,personalloan_bad,personalloan_good
0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,3,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


This section seems to be an indication of the number of credit accounts by category (sum accross column matches with 'totalaccounts' in further section).

I would assume good and bad refer to Good and Bad Credit which would definitely give an indication of credit risk.

We already have info on the sum in other section and I don't think averaging across all columns makes any sense. Instead we can focus on aggregate values on good and bad credits.
- Max Good / Bad
- Sum of Good / Bad



In [8]:
def account_rating(data):
    
    # read data into Dataframe with type int
    df = pd.DataFrame(list(data['data_consumerfullcredit_accountrating'])).astype(int)

    # Clean names
    names_cleaning = {'noof': '', 'accounts': '_', 'ccounts': '_' }
    for value, key in names_cleaning.items():
         df.columns = df.columns.str.replace(value, key)     

    # feature engineering    
    df['max_good'] = df.loc[:, [col for col in df.columns if col.endswith('_good')]].min(axis=1).round(2)
    df['max_bad'] = df.loc[:, [col for col in df.columns if col.endswith('_bad')]].min(axis=1).round(2)
    df['sum_good'] = df.loc[:, [col for col in df.columns if col.endswith('_good')]].sum(axis=1).round(2)
    df['sum_bad'] = df.loc[:, [col for col in df.columns if col.endswith('_bad')]].sum(axis=1).round(2)
        
    return df.add_prefix('rating_')

In [9]:
account_rating(df)

,rating_other_bad,rating_other_good,rating_retail_bad,rating_retail_good,rating_telecom_bad,rating_autoloan_bad,rating_autoloan_good,rating_homeloan_bad,rating_telecom_good,rating_homeloan_good,...,rating_creditcard_bad,rating_jointloan_good,rating_studyloan_good,rating_creditcard_good,rating_personalloan_bad,rating_personalloan_good,rating_max_good,rating_max_bad,rating_sum_good,rating_sum_bad
0,0,3,0,2,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,7,0
1,0,3,0,12,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,17,0
2,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,1


---

### Enquiry Details

In [10]:
enquiry_details = pd.DataFrame(list(df['data_consumerfullcredit_enquirydetails']))
enquiry_details.columns = enquiry_details.columns.str.replace('subscriberenquiry', '')
enquiry_details.head()

,productid,matchingrate,engineid,resultid
0,45,90,5012874225,6381470
1,45,90,5012883736,6383644
2,45,90,5012883827,6383655


These data seems to be related to the query itself. I don't think these data would bring any value.

---

### Guarantor information

In [11]:
guarantor_count = pd.DataFrame(list(df['data_consumerfullcredit_guarantorcount']))
guarantor_count.head()

,accounts,guarantorssecured
0,0,0
1,0,0
2,0,0


In [12]:
guarantor_details = pd.DataFrame(list(df['data_consumerfullcredit_guarantordetails']))
guarantor_details.head()

,guarantorgender,guarantorotherid,guarantoraddress1,guarantoraddress2,guarantoraddress3,guarantorpassport,guarantorfirstname,guarantorothername,guarantordateofbirth,guarantornationalidno,guarantorhometelephone,guarantorworktelephone,guarantordriverlicenceno,guarantormobiletelephone
0,None,None,None,None,None,None,None,None,1900-01-01T00:00:00+01:00,None,None,None,None,None
1,None,None,None,None,None,None,None,None,1900-01-01T00:00:00+01:00,None,None,None,None,None
2,None,None,None,None,None,None,None,None,1900-01-01T00:00:00+01:00,None,None,None,None,None


In that section, we have information on potential guarantors for credit applicant. 

There is actually no guarantor data in that small sample but that doesn't mean it could not be a useful information. The presence of a guarantor is potentially a risk mitigant. 

Potential interesting features:

- Guarantors count
- Guarantors secured (not sure what it is)
- Age difference between Guarantor and applicant

Other features that would require more samples to explore:
- Do they have same last name (binary indicator) - not sure whether last name available in there
- Same nationality ? - not sure what passport field contains

In [13]:
def guarantor_info(data, ref_date = TODAY):
        
    # read data into Dataframe
    df = pd.concat([pd.DataFrame(list(data['data_consumerfullcredit_guarantorcount'])),
                    pd.DataFrame(list(data['data_consumerfullcredit_guarantordetails']))], axis = 1)

    # compute age (use -1 if date is 1900-01-01)
    df['guarantordateofbirth'] = df['guarantordateofbirth'].apply(lambda x: datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10])))  
    df['guarantor_age'] = df['guarantordateofbirth'].apply(lambda x : -1 if (x.year == 1900) else relativedelta.relativedelta(ref_date, x).years) 

    # keep useful columns only and adjust data type
    useful_cols = ['accounts','guarantorssecured', 'guarantor_age'] # 'guarantorpassport', 'guarantorfirstname', 'guarantordateofbirth' 
    df = df.loc[:, useful_cols].astype(int)

    # rename columns
    df.columns = ['guarantors_count','guarantors_secured', 'guarantor_age']
        
    return df

In [14]:
guarantor_info(df)

,guarantors_count,guarantors_secured,guarantor_age
0,0,0,-1
1,0,0,-1
2,0,0,-1


---

### Employment History

In [15]:
list(df['data_consumerfullcredit_employmenthistory'])

[[{'occupation': 'PUBLIC SERVANTS',
   'updatedate': '16/09/2016',
   'updateondate': '16/09/2016',
   'employerdetail': 'ALL MILITARY STAFFS'},
  {'occupation': 'PUBLIC SERVANTS'},
  {'occupation': 'PUBLIC SERVANTS'},
  {'occupation': 'Army Police Defence', 'employerdetail': 'Nigerian navy'},
  {'occupation': 'PUBLIC SERVANTS'}],
 [{'occupation': 'CIVIL SERVANT',
   'updatedate': '21/11/2014',
   'updateondate': '21/11/2014',
   'employerdetail': None},
  {'occupation': 'DOCTOR',
   'employerdetail': 'MINISTRY OF WOMEN AFFAIRS AND SOCIAL DEVELOPMENT ABIA'},
  {'occupation': 'DOCTOR',
   'employerdetail': 'MINISTRY OF WOMEN AFFAIRS AND SOCIAL DEVELOPMENT'},
  {'occupation': 'DOCTOR',
   'employerdetail': 'MINISTRY OF WOMEN AFFAIRS AND SOCIAL DEVELOPMENT'},
  {'occupation': 'DOCTOR'}],
 [{'occupation': 'STUDENT'},
  {'occupation': 'STUDENT'},
  {'occupation': 'STUDENT'},
  {'occupation': 'STUDENT'},
  {'occupation': 'STUDENT'}]]

It seems that this category provides information on the last 5 updates regarding the applicant's occupation. 

For each update, we can have different level of details (occupation, employer, updatedate).

Kind of features we could extract from this:
- Number of different occupations in last 5 updates
- Any status of unemployment (Binary indicator)
- Is student ?
- Time to last update (if any date)

Other features that would require more knowledge to understand how to encode: 
- Last known occupation - requires better knowledge of data for potential segmentation or to map specific occupations
- First known occupation - requires better knowledge of data for potential segmentation or to map specific occupations
- Last known employer - requires better knowledge of data for potential segmentation or to map specific occupations

In [16]:
def employment_history(data, ref_date=TODAY):
        
    # Read data into Dataframe
    df = pd.DataFrame(data['data_consumerfullcredit_employmenthistory']).rename(columns = {'data_consumerfullcredit_employmenthistory':'employment_raw'})

    df['nb_occupations'] = df['employment_raw'].apply(lambda x : len(set([item['occupation'] for item in x])))
    df['is_student'] =  df['employment_raw'].apply(lambda x : int(x[0]['occupation'].lower() == 'student'))
    df['unemployment'] =  df['employment_raw'].apply(lambda x : 1 if 'unemployed' in [item['occupation'].lower() for item in x] else 0)

    # Compute time to last_update        
    df['last_update_date'] = pd.to_datetime(df['employment_raw'].apply(lambda x : x[0]['updatedate'] if x[0].get('updatedate') else 0), format='%d/%m/%Y', errors = 'coerce')
    df['time_to_last_update'] = df['last_update_date'].apply(lambda x : (ref_date - x).days).fillna(-1).astype(int)

    # keep useful columns
    df = df.drop(['employment_raw', 'last_update_date'], axis = 1)                          
                                         
    return df.add_prefix('employment_')

In [17]:
employment_history(df)

,employment_nb_occupations,employment_is_student,employment_unemployment,employment_time_to_last_update
0,2,0,0,2338
1,2,0,0,3003
2,1,1,0,-1


---

### Enquiry history

In [18]:
list(df['data_consumerfullcredit_enquiryhistorytop'])[0]

[{'daterequested': '06/08/2020 18:05:36',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresultid': '6050316'},
 {'daterequested': '01/08/2020 06:59:09',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresultid': '5983396'},
 {'daterequested': '08/07/2020 15:58:44',
  'enquiryreason': 'application of existing credit by a borrower',
  'subscribername': 'NewEdge Finance Limited  - transsnet',
  'subscriberenquiryresultid': '5680296'},
 {'daterequested': '03/07/2020 22:58:39',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresultid': '5637826'},
 {'daterequested': '04/06/2020 07:55:58',
  'enquiryreason': 'Credit scoring of the client by credit bureau',
  'subscribername': 'Fairmoney - FairMoneyAPI',
  'subscriberenquiryresulti

This category provides an exhaustive list of enquiries made to the provider on the applicant with:
- date of enquiry:
- name of subscriber who made the enquiry
- reason of the enquiry - seems like you're using a credit bureau score for your model :)

Examples of features:
- Number of recent enquiries made by subscribers (other than Fairmoney) could be an indicator of a risk. 
- It could be split by type of subscriber (banks vs others).
- The type of enquiries made could also be a useful indicator (I see that there are enquiries regarding credit application).

In [19]:
def enquiry_history(data, ref_date=TODAY):
        
    # Read data into Dataframe
    df = pd.DataFrame(data['data_consumerfullcredit_enquiryhistorytop']).rename(columns = {'data_consumerfullcredit_enquiryhistorytop':'enquiries'})        

    # compute nb of enquiries from other companies in last 12 and 6 months 
    ref_date_12m = ref_date - relativedelta.relativedelta(years=1)
    ref_date_6m = ref_date - relativedelta.relativedelta(months=6)      

    df['nb_last_12M'] = df['enquiries'].apply(lambda x : len([item for item in x if not 'fairmoney' in item['subscribername'].lower()  and pd.to_datetime(item['daterequested'], format = '%d/%m/%Y %H:%M:%S') > ref_date_12m]))
    df['nb_last_6M'] = df['enquiries'].apply(lambda x : len([item for item in x if not 'fairmoney' in item['subscribername'].lower()  and pd.to_datetime(item['daterequested'], format = '%d/%m/%Y %H:%M:%S') > ref_date_6m]))

    # compute time to last enquiry 
    df['last_enquiry_date'] = df['enquiries'].apply(lambda x : [pd.to_datetime(item['daterequested'], format = '%d/%m/%Y %H:%M:%S') for item in x if not 'fairmoney' in item['subscribername'].lower()][0])
    df['time_to_last_enquiry'] = df['last_enquiry_date'].apply(lambda x : (ref_date - x).days)

    # keep useful columns
    df = df.drop(['enquiries','last_enquiry_date'], axis = 1)                          
        
    return df.add_prefix('enquiries_')

In [20]:
enquiry_history(df, ref_date = datetime.datetime(2021,1,1))

,enquiries_nb_last_12M,enquiries_nb_last_6M,enquiries_time_to_last_enquiry
0,1,1,176
1,4,0,190
2,2,0,229


---

### Delinquency data

In [21]:
credit_account_summary = pd.DataFrame(list(df['data_consumerfullcredit_creditaccountsummary']))
credit_account_summary.head()

,rating,amountarrear,amountarrear1,totalaccounts,totalaccounts1,lastjudgementdate,lastjudgementdate1,totalaccountarrear,totalaccountarrear1,totaljudgementamount,...,totaldishonouredamount,totalmonthlyinstalment,totalnumberofjudgement,totaldishonouredamount1,totalmonthlyinstalment1,totalnumberofjudgement1,totalnumberofdishonoured,totalnumberofdishonoured1,totalaccountingodcondition,totalaccountingodcondition1
0,13,"24,041.00",0.00,7,0,-,-,2,0,0,...,0.00,"77,404.00",0,0.00,0.00,0,0,0,0,0
1,2,0.00,0.00,17,0,-,-,1,0,0,...,0.00,"132,176.00",0,0.00,0.00,0,0,0,0,0
2,109,"12,000.00",0.00,3,0,-,-,1,0,0,...,0.00,"7,000.00",0,0.00,0.00,0,0,0,0,0


In [22]:
delinquency_information = pd.DataFrame(list(df['data_consumerfullcredit_deliquencyinformation']))
delinquency_information.head()

,accountno,periodnum,subscribername,monthsinarrears
0,2150224012,20140930,First City Monument Bank Ltd Lagos,13
1,XXX,20170430,First City Monument Bank Ltd Lagos,2
2,4181042465,20200630,Branch International Financial Services Limited,109


These two classes of data contain information on potential previous issues of payments on known credit accounts (past debts, nb of month in arrears, amount of arrears, judgments, dishonoured cheques). This is obviously an interesting indicator of risk.

Potential features:
- total_credit_accounts
- total_account_arrear - accounts with arrear - could be expressed in percentage of total
- nb of months in arrears
- total_nb_of_judgment
- total_nb_of__dishonoured
- amnt_arrear 
- total_judgment_debt 
- total__monthly_instalment 
- time to periodnum (probably the last date of arrears)

Other features that would require more knowledge to engineer: 
- type of subscriber - requires encoding / depends on cardinality
- Time to last judgment date - no format example

In [24]:
def delinquency(data, ref_date=TODAY):
        
    # load data into Dataframe
    df = pd.concat([ pd.DataFrame(list(data['data_consumerfullcredit_creditaccountsummary'])),
                     pd.DataFrame(list(data['data_consumerfullcredit_deliquencyinformation']))], axis=1) 

    # Clean names
    names_cleaning = {'numberof':'_nb_of_', 'amount':'amnt', 'total': 'total_', 'judgement': '_judgment_', 'outstanding': '_judgment_', 'monthly': '_monthly_', 'arrear': '_arrear', 'dishonoured': '_dishonoured_', 'accountingod': 'accountingod_'}
    for value, key in names_cleaning.items():
         df.columns = df.columns.str.replace(value, key)

    # convert numeric columns to int
    int_cols = ['total_accounts', 'total_account_arrear', 'monthsin_arrears', 'amnt_arrear', 'total__nb_of__dishonoured_', 'total__dishonoured_amnt', 'total__nb_of__judgment_', 'total__judgment_debt', 'total__monthly_instalment']
    df[int_cols] = df[int_cols].replace(',', '', regex=True).astype(float).astype(int)

    # convert periodnum to datetime format
    df['periodnum'] = df['periodnum'].apply(lambda x: datetime.datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
    df['time_to_periodnum'] = df['periodnum'].apply(lambda x : (ref_date - x).days)

    # compute share of accounts with arrears
    df['accounts_arrear_pct'] =  (df['total_account_arrear'] /  df['total_accounts'] * 100).astype(int)


    # keep useful columns
    useful_cols = ['total_accounts', 'total_account_arrear', 'accounts_arrear_pct', 'monthsin_arrears', 'amnt_arrear', 'total__nb_of__dishonoured_', 'total__dishonoured_amnt', 'total__nb_of__judgment_', 'total__judgment_debt', 'total__monthly_instalment', 'time_to_periodnum']
    df = df.loc[:, useful_cols]

    return df

In [25]:
delinquency(df)

,total_accounts,total_account_arrear,accounts_arrear_pct,monthsin_arrears,amnt_arrear,total__nb_of__dishonoured_,total__dishonoured_amnt,total__nb_of__judgment_,total__judgment_debt,total__monthly_instalment,time_to_periodnum
0,7,2,28,13,24041,0,0,0,105435,77404,3055
1,17,1,5,2,0,0,0,0,294770,132176,2112
2,3,1,33,109,12000,0,0,0,110919,7000,955


---

### Credit Agreement

In [148]:
list(df['data_consumerfullcredit_creditagreementsummary'])[0][0]

{'currency': 'NGN',
 'accountno': '0324562636176001',
 'closeddate': '01/11/2020',
 'loanduration': '1170',
 'subaccountno': None,
 'accountstatus': 'Open',
 'amountoverdue': '22,441.39',
 'subscribername': 'First City Monument Bank Ltd Lagos',
 'lastupdateddate': '07/08/2020',
 'instalmentamount': '24,368.39',
 'currentbalanceamt': '82,733.58',
 'dateaccountopened': '15/01/2016',
 'openingbalanceamt': '814,700.00',
 'performancestatus': 'Performing',
 'repaymentfrequency': 'Monthly',
 'indicatordescription': 'Personal secured loan'}

This section contains further details on each known credit accounts but we can find the same information and more in the monthly payment history section that we will cover below.

---

### Personal Details

In [27]:
personal_details = pd.DataFrame(list(df['data_consumerfullcredit_personaldetailssummary']))
personal_details.head()

,gender,header,surname,birthdate,firstname,otheridno,cellularno,consumerid,dependants,othernames,...,postaladdress4,hometelephoneno,propertyownedtype,bankverificationno,residentialaddress1,residentialaddress2,residentialaddress3,residentialaddress4,worktelephoneno,passportno
0,Male,PERSONAL DETAILS SUMMARY: XXX,XXX,06/05/1991,XXX,None,XXX,128566,0,XXX,...,"{'xml:space': 'preserve', '__content__': ' '}",XXX,None,XXX,XXX,KADUNA STATE NIGERIA,None,"{'xml:space': 'preserve', '__content__': ' '}",NaN,NaN
1,Female,PERSONAL DETAILS SUMMARY: XXX,XXX,30/11/1985,XXX,None,07062688658,58793,0,XXX,...,"{'xml:space': 'preserve', '__content__': ' '}",XXX,None,221XXX82472688,XXX,UMUAHIA,None,"{'xml:space': 'preserve', '__content__': ' '}",XXX,NaN
2,Female,PERSONAL DETAILS SUMMARY: XXX,XXX,14/07/1983,XXX,None,XXX,17688366,0,XXX,...,NaN,NaN,None,XXX,Nigeria,Nigeria,None,Nigeria,NaN,XXX


This section contains personal information on the applicant

Potential interesting features could be: 

- gender - is_male
- age (from birthdate)
- nb of dependants
- nationality (encoded)

Other features that would require more examples to definer engineering:
- propertyownedtype
- employer_type
- city - state
- area of address
- postal vs residential
    

In [28]:
def personal(data, ref_date=TODAY):
        
    # load data into Dataframe with type int
    df = pd.DataFrame(list(data['data_consumerfullcredit_personaldetailssummary']))

    # compute age
    df['birthdate'] = pd.to_datetime(df['birthdate'], format='%d/%m/%Y', errors='coerce')
    df['age'] = df['birthdate'].apply(lambda x : relativedelta.relativedelta(ref_date, x).years) 

    # compute is_male
    df['is_male'] = (df['gender'].str.lower() == 'male').astype(int)

    # keep useful columns
    useful_cols = ['is_male', 'age', 'dependants', 'nationality']
    df = df.loc[:, useful_cols]
        
    return df

In [29]:
personal(df)

,is_male,age,dependants,nationality
0,1,31,0,Nigeria
1,0,37,0,Nigeria
2,0,39,0,Nigeria


---

### Monthly payment history

In [162]:
list(df['data_consumerfullcredit_accountmonthlypaymenthistory'])[0][2]

{'m01': '#',
 'm02': '#',
 'm03': '#',
 'm04': '#',
 'm05': '#',
 'm06': '#',
 'm07': '#',
 'm08': '#',
 'm09': '#',
 'm10': '#',
 'm11': '#',
 'm12': '#',
 'm13': '#',
 'm14': '#',
 'm15': '#',
 'm16': '#',
 'm17': '#',
 'm18': '#',
 'm19': '#',
 'm20': '#',
 'm21': '#',
 'm22': '#',
 'm23': '#',
 'm24': '#',
 'header': 'Details of Credit Agreement with "First City Monument Bank Ltd Lagos" for Account Number: 2150224012',
 'currency': 'NGN',
 'accountno': '2150224012',
 'tablename': 'Consumer24MonthlyPayment',
 'accountnote': None,
 'displaytext': 'Consumer 24 Monthly Payment',
 'loanduration': 'Not Available',
 'subaccountno': None,
 'accountstatus': 'WrittenOff',
 'amountoverdue': '1,478.37',
 'subscribername': 'First City Monument Bank Ltd Lagos',
 'lastupdateddate': '01/03/2018',
 'currentbalanceamt': '938.00',
 'dateaccountopened': '09/05/2013',
 'openingbalanceamt': '0.00',
 'performancestatus': 'Performing',
 'subscribertypeind': 'C',
 'indicatordescription': 'Overdraft',
 'cur

In [31]:
list(df['data_consumerfullcredit_accountmonthlypaymenthistoryheader'])

[{'mh01': '2020\nAUG',
  'mh02': '2020\nJUL',
  'mh03': '2020\nJUN',
  'mh04': '2020\nMAY',
  'mh05': '2020\nAPR',
  'mh06': '2020\nMAR',
  'mh07': '2020\nFEB',
  'mh08': '2020\nJAN',
  'mh09': '2019\nDEC',
  'mh10': '2019\nNOV',
  'mh11': '2019\nOCT',
  'mh12': '2019\nSEP',
  'mh13': '2019\nAUG',
  'mh14': '2019\nJUL',
  'mh15': '2019\nJUN',
  'mh16': '2019\nMAY',
  'mh17': '2019\nAPR',
  'mh18': '2019\nMAR',
  'mh19': '2019\nFEB',
  'mh20': '2019\nJAN',
  'mh21': '2018\nDEC',
  'mh22': '2018\nNOV',
  'mh23': '2018\nOCT',
  'mh24': '2018\nSEP',
  'company': 'Company',
  'tablename': 'Consumer24MonthlyPaymentHeader',
  'displaytext': 'Consumer 24 Monthly Payment Header'},
 {'mh01': '2020\nSEP',
  'mh02': '2020\nAUG',
  'mh03': '2020\nJUL',
  'mh04': '2020\nJUN',
  'mh05': '2020\nMAY',
  'mh06': '2020\nAPR',
  'mh07': '2020\nMAR',
  'mh08': '2020\nFEB',
  'mh09': '2020\nJAN',
  'mh10': '2019\nDEC',
  'mh11': '2019\nNOV',
  'mh12': '2019\nOCT',
  'mh13': '2019\nSEP',
  'mh14': '2019\nAUG

This section contains further details on each known credit accounts, including:

- Details of last 24 monthly payments (a list of amounts and a list of months)
- loan duration in days
- Status: Open or closed
- opening and closing date (if closed)
- overdue amount - we have the total amount in previous section (arrear)
- subscribername -> creditor
- subscribertype -> C for credit, 
- Opening Balance = Amount borrowed
- Indicator description : Installment account or secured loan
- Current Balance = Opening Balanced - paid installments
- Current Balance Debit Ind : Credit or Debit
- monthly instalment amount = Amount to be paid on repayment frequency
- repaymentfrequency : frequency of payments

Note: There are cases of overdraft which are not requested loans but debit in checking account I guess

Looking at many cases, this data is pretty unstructured and would require significant level of analysis and/or specific business knowledge.

I will stick with basic features here:

- average / max monthly installment
- average / max overdue amount
- average / max / sum loan duration in days
- nb of open accounts
- nb of overdraft occurences

In [175]:
def payment_history(data, ref_date=TODAY):
        
    # load data into Dataframe
    df = pd.DataFrame(data['data_consumerfullcredit_accountmonthlypaymenthistory'])
    df.columns = ['payment_history']
    
    # compute average and max monthly installment
    df['installments'] = df['payment_history'].apply(lambda x: [float(item['monthlyinstalmentamt'].replace(',', '')) for item in x if item.get('monthlyinstalmentamt')])
    df['avg_installment'] = df['installments'].apply(lambda x : int(np.mean(x)))
    df['max_installment'] = df['installments'].apply(lambda x : int(np.max(x)))
    
    # compute average and max overdue amount
    df['amnt_overdue'] = df['payment_history'].apply(lambda x: [float(item['amountoverdue'].replace(',', '')) for item in x if item.get('amountoverdue')])
    df['avg_overdue'] = df['amnt_overdue'].apply(lambda x : int(np.mean(x)))
    df['max_overdue'] = df['amnt_overdue'].apply(lambda x : int(np.max(x)))
    
    # compute average / max / sum loan duration
    df['loan_duration'] = df['payment_history'].apply(lambda x: [int(item['loanduration'].replace(' Day(s)', '')) for item in x if item.get('loanduration') and item['loanduration']!='Not Available'])
    df['avg_duration'] = df['loan_duration'].apply(lambda x : int(np.mean(x)))
    df['max_duration'] = df['loan_duration'].apply(lambda x : int(np.max(x)))
    df['sum_duration'] = df['loan_duration'].apply(lambda x : int(np.sum(x)))
    
    # compute nb of open accounts
    df['nb_open'] = df['payment_history'].apply(lambda x: np.sum([1 for item in x if item.get('accountstatus') and item['accountstatus']=='Open']))
    
    # compute nb of overdraft occurences
    df['nb_overdraft'] = df['payment_history'].apply(lambda x: int(np.sum([1 for item in x if item.get('indicatordescription') and item['indicatordescription']=='Overdraft'])))
    
    # keep useful columns
    drop_cols = ['payment_history', 'installments','amnt_overdue', 'loan_duration']
    df = df.drop(drop_cols, axis =1)

    return df.add_prefix('payment_hist_')

In [176]:
payment_history(df)

,payment_hist_avg_installment,payment_hist_max_installment,payment_hist_avg_overdue,payment_hist_max_overdue,payment_hist_avg_duration,payment_hist_max_duration,payment_hist_sum_duration,payment_hist_nb_open,payment_hist_nb_overdraft
0,19351,24368,3434,22441,1775,5843,7103,3,2
1,8260,13722,0,0,414,1830,2072,4,0
2,2333,7000,4000,12000,187,374,375,3,0


---

### Identification history

In [33]:
list(df['data_consumerfullcredit_identificationhistory'])

[nan,
 [{'updatedate': '21/11/2014',
   'updateondate': '21/11/2014',
   'identificationtype': 'National ID Number',
   'identificationnumber': 'XXX'},
  {'updatedate': '17/01/2017',
   'updateondate': '17/01/2017',
   'identificationtype': 'National ID Number',
   'identificationnumber': 'XXX'}],
 nan]

This category contains a track of ID document updates. 

I'm not sure whether this could be useful but we could build a couple of indicators from this:
- Any record of ID 
- Nb of ID updates
- Time to last update


In [34]:
def identification(data, ref_date=TODAY):
     
    # load data into Dataframe
    df = pd.DataFrame(data['data_consumerfullcredit_identificationhistory'])
    df.columns = ['identification']

    # # # compute indicators
    df['has_id'] = df['identification'].notnull().astype(int)
    df['nb_updates'] = df.loc[df['identification'].notnull(),'identification'].apply(lambda x : len(x))
    df['nb_updates'] = df['nb_updates'].fillna(0)
    df['last_update'] = df.loc[df['identification'].notnull(),'identification'].apply(lambda x : x[0]['updatedate'])
    df['last_update'] = pd.to_datetime(df['last_update'], format = '%d/%m/%Y', errors = 'coerce')
    df['time_to_update'] = df.loc[df['identification'].notnull(), 'last_update'].apply(lambda x : relativedelta.relativedelta(ref_date, x).days)
    df['time_to_update'] = df['time_to_update'].fillna(-1)

    # keep useful columns
    useful_cols = ['has_id', 'nb_updates', 'time_to_update']
    df = df.loc[:, useful_cols]                                
        
    return df

In [35]:
personal(df)

,is_male,age,dependants,nationality
0,1,31,0,Nigeria
1,0,37,0,Nigeria
2,0,39,0,Nigeria


----

## Feature Extraction Pipeline

Let's build a global extraction class from all these functions.

---

In [229]:
import pandas as pd
import datetime
from dateutil import relativedelta
import json
import numpy as np

TODAY = datetime.datetime.today()


class CreditReport:
    """Feature Extraction from Credit Reports."""

    def __init__(self, json_path: str, ids: list[int] = None, ref_date: datetime = TODAY):
        self.json_path = json_path
        self.ids = ids
        self.ref_date = ref_date

    def load_data(self):
        """Load Json file from path."""
        with open(self.json_path) as f:
            data = json.load(f)

        df = pd.json_normalize(data, max_level=2, sep='_', record_prefix=False)

        if self.ids:

            for app_id in self.ids:
                if not isinstance(app_id, int):
                    raise ValueError("applicant ids should be integers")

            df = df.loc[df['application_id'].isin(self.ids), :]

        return df

    def account_categories(self, data):
        """Credit category features."""

        # load data
        df = pd.DataFrame(list(data['data_consumerfullcredit_accountrating'])).astype(int)

        # Clean names
        names_cleaning = {'noof': '', 'accounts': '_', 'ccounts': '_'}
        for value, key in names_cleaning.items():
            df.columns = df.columns.str.replace(value, key)

        # feature engineering
        df['max_good'] = df.loc[:, [col for col in df.columns if col.endswith('_good')]].min(axis=1).round(2)
        df['max_bad'] = df.loc[:, [col for col in df.columns if col.endswith('_bad')]].min(axis=1).round(2)
        df['sum_good'] = df.loc[:, [col for col in df.columns if col.endswith('_good')]].sum(axis=1).round(2)
        df['sum_bad'] = df.loc[:, [col for col in df.columns if col.endswith('_bad')]].sum(axis=1).round(2)

        return df.add_prefix('rating_')

    def guarantor_info(self, data):
        """Guarantor features."""

        # Load data
        df = pd.concat([pd.DataFrame(list(data['data_consumerfullcredit_guarantorcount'])),
                        pd.DataFrame(list(data['data_consumerfullcredit_guarantordetails']))], axis=1)

        # compute age (use -1 if date is 1900-01-01)
        df['guarantordateofbirth'] = df['guarantordateofbirth'].apply(
            lambda x: datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10])))
        df['guarantor_age'] = df['guarantordateofbirth'].apply(
            lambda x: -1 if (x.year == 1900) else relativedelta.relativedelta(self.ref_date, x).years)

        # keep useful columns only and adjust data type
        useful_cols = ['accounts', 'guarantorssecured', 'guarantor_age']
        df = df.loc[:, useful_cols].astype(int)

        # rename columns
        df.columns = ['guarantors_count', 'guarantors_secured', 'guarantor_age']

        return df

    def employment_history(self, data):
        """Employment history features."""

        # Load data
        df = pd.DataFrame(data['data_consumerfullcredit_employmenthistory']).rename(
            columns={'data_consumerfullcredit_employmenthistory': 'employment_raw'})

        df['nb_occupations'] = df['employment_raw'].apply(lambda x: len(set([item['occupation'] for item in x])))
        df['is_student'] = df['employment_raw'].apply(lambda x: int(x[0]['occupation'].lower() == 'student'))
        df['unemployment'] = df['employment_raw'].apply(lambda x: 1 if 'unemployed' in [item['occupation'].lower() for item in x] else 0)

        # Compute time to last_update
        df['last_update_date'] = pd.to_datetime( df['employment_raw'].apply(lambda x: x[0]['updatedate'] if x[0].get('updatedate') else 0), format='%d/%m/%Y', errors='coerce')
        df['time_to_last_update'] = df['last_update_date'].apply(lambda x: (self.ref_date - x).days).fillna(-1).astype(int)

        # keep useful columns
        df = df.drop(['employment_raw', 'last_update_date'], axis=1)

        return df.add_prefix('employment_')

    def enquiry_history(self, data):
        """Employment history features."""

        # Load data
        df = pd.DataFrame(data['data_consumerfullcredit_enquiryhistorytop']).rename(
            columns={'data_consumerfullcredit_enquiryhistorytop': 'enquiries'})

        # compute nb of enquiries from other companies in last 12 and 6 months
        ref_date_12m = self.ref_date - relativedelta.relativedelta(years=1)
        ref_date_6m = self.ref_date - relativedelta.relativedelta(months=6)

        df['nb_last_12M'] = df['enquiries'].apply(lambda x: len([item for item in x if not 'fairmoney' in item[
            'subscribername'].lower() and pd.to_datetime(item['daterequested'], format='%d/%m/%Y %H:%M:%S') > ref_date_12m]))
        df['nb_last_6M'] = df['enquiries'].apply(lambda x: len([item for item in x if not 'fairmoney' in item[
            'subscribername'].lower() and pd.to_datetime(item['daterequested'], format='%d/%m/%Y %H:%M:%S') > ref_date_6m]))

        # compute time to last enquiry
        df['last_enquiry_date'] = df['enquiries'].apply(lambda x:
                                                        [pd.to_datetime(item['daterequested'], format='%d/%m/%Y %H:%M:%S')
                                                         for item in x if
                                                         not 'fairmoney' in item['subscribername'].lower()][0])
        df['time_to_last_enquiry'] = df['last_enquiry_date'].apply(lambda x: (self.ref_date - x).days)

        # keep useful columns
        df = df.drop(['enquiries', 'last_enquiry_date'], axis=1)

        return df.add_prefix('enquiries_')

    def delinquency(self, data):
        """Deliquency features."""
        # load data into Dataframe
        df = pd.concat([pd.DataFrame(list(data['data_consumerfullcredit_creditaccountsummary'])),
                        pd.DataFrame(list(data['data_consumerfullcredit_deliquencyinformation']))], axis=1)

        # Clean names
        names_cleaning = {'numberof': '_nb_of_', 'amount': 'amnt', 'total': 'total_', 'judgement': '_judgment_',
                          'outstanding': '_judgment_', 'monthly': '_monthly_', 'arrear': '_arrear',
                          'dishonoured': '_dishonoured_', 'accountingod': 'accountingod_'}
        for value, key in names_cleaning.items():
            df.columns = df.columns.str.replace(value, key)

        # convert numeric columns to int
        int_cols = ['total_accounts', 'total_account_arrear', 'monthsin_arrears', 'amnt_arrear',
                    'total__nb_of__dishonoured_', 'total__dishonoured_amnt', 'total__nb_of__judgment_',
                    'total__judgment_debt', 'total__monthly_instalment']
        df[int_cols] = df[int_cols].replace(',', '', regex=True).astype(float).astype(int)

        # convert periodnum to datetime format
        df['periodnum'] = df['periodnum'].apply(lambda x: datetime.datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
        df['time_to_periodnum'] = df['periodnum'].apply(lambda x: (self.ref_date - x).days)

        # compute share of accounts with arrears
        df['accounts_arrear_pct'] = (df['total_account_arrear'] / df['total_accounts'] * 100).astype(int)

        # keep useful columns
        useful_cols = ['total_accounts', 'total_account_arrear', 'accounts_arrear_pct', 'monthsin_arrears', 'amnt_arrear',
                       'total__nb_of__dishonoured_', 'total__dishonoured_amnt', 'total__nb_of__judgment_',
                       'total__judgment_debt', 'total__monthly_instalment', 'time_to_periodnum']
        df = df.loc[:, useful_cols]

        return df

    def personal(self, data):
        """Personal data features."""
        # load data into Dataframe with type int
        df = pd.DataFrame(list(data['data_consumerfullcredit_personaldetailssummary']))

        # compute age
        df['birthdate'] = pd.to_datetime(df['birthdate'], format='%d/%m/%Y', errors='coerce')
        df['age'] = df['birthdate'].apply(lambda x: relativedelta.relativedelta(self.ref_date, x).years)

        # compute is_male
        df['is_male'] = (df['gender'].str.lower() == 'male').astype(int)

        # keep useful columns
        useful_cols = ['is_male', 'age', 'dependants', 'nationality']
        df = df.loc[:, useful_cols]

        return df

    def payment_history(self, data):
        """Payment history features."""
        # load data into Dataframe
        df = pd.DataFrame(data['data_consumerfullcredit_accountmonthlypaymenthistory'])
        df.columns = ['payment_history']

        # compute average and max monthly installment
        df['installments'] = df['payment_history'].apply(
            lambda x: [float(item['monthlyinstalmentamt'].replace(',', '')) for item in x if
                       item.get('monthlyinstalmentamt')])
        df['avg_installment'] = df['installments'].apply(lambda x: int(np.mean(x)))
        df['max_installment'] = df['installments'].apply(lambda x: int(np.max(x)))

        # compute average and max overdue amount
        df['amnt_overdue'] = df['payment_history'].apply(
            lambda x: [float(item['amountoverdue'].replace(',', '')) for item in x if item.get('amountoverdue')])
        df['avg_overdue'] = df['amnt_overdue'].apply(lambda x: int(np.mean(x)))
        df['max_overdue'] = df['amnt_overdue'].apply(lambda x: int(np.max(x)))

        # compute average / max / sum loan duration
        df['loan_duration'] = df['payment_history'].apply(
            lambda x: [int(item['loanduration'].replace(' Day(s)', '')) for item in x if
                       item.get('loanduration') and item['loanduration'] != 'Not Available'])
        df['avg_duration'] = df['loan_duration'].apply(lambda x: int(np.mean(x)))
        df['max_duration'] = df['loan_duration'].apply(lambda x: int(np.max(x)))
        df['sum_duration'] = df['loan_duration'].apply(lambda x: int(np.sum(x)))

        # compute nb of open accounts
        df['nb_open'] = df['payment_history'].apply(
            lambda x: np.sum([1 for item in x if item.get('accountstatus') and item['accountstatus'] == 'Open']))

        # compute nb of overdraft occurences
        df['nb_overdraft'] = df['payment_history'].apply(lambda x: int(np.sum(
            [1 for item in x if item.get('indicatordescription') and item['indicatordescription'] == 'Overdraft'])))

        # keep useful columns
        drop_cols = ['payment_history', 'installments', 'amnt_overdue', 'loan_duration']
        df = df.drop(drop_cols, axis=1)

        return df.add_prefix('payment_hist_')

    def identification(self, data):
        """Identification features."""
        # load data into Dataframe
        df = pd.DataFrame(data['data_consumerfullcredit_identificationhistory'])
        df.columns = ['identification']

        # # # compute indicators
        df['has_id'] = df['identification'].notnull().astype(int)
        df['nb_updates'] = df.loc[df['identification'].notnull(), 'identification'].apply(lambda x: len(x))
        df['nb_updates'] = df['nb_updates'].fillna(0)
        df['last_update'] = df.loc[df['identification'].notnull(), 'identification'].apply(lambda x: x[0]['updatedate'])
        df['last_update'] = pd.to_datetime(df['last_update'], format='%d/%m/%Y', errors='coerce')
        df['time_to_update'] = df.loc[df['identification'].notnull(), 'last_update'].apply(
            lambda x: relativedelta.relativedelta(self.ref_date, x).days)
        df['time_to_update'] = df['time_to_update'].fillna(-1)

        # keep useful columns
        useful_cols = ['has_id', 'nb_updates', 'time_to_update']
        df = df.loc[:, useful_cols]

        return df
    
    def specific_features(self, extract_function):
        data = self.load_data()
        features = pd.concat([
            pd.DataFrame(data['application_id']),
            extract_function(data)],
            axis=1
        )

        return features

    def all_features(self):
        """Get all features."""
        data = self.load_data()
        features = pd.concat([
            pd.DataFrame(data['application_id']),
            self.account_categories(data),
            self.guarantor_info(data),
            self.employment_history(data),
            self.enquiry_history(data),
            self.delinquency(data),
            self.payment_history(data),
            self.personal(data),
            self.identification(data)],
            axis=1
        )

        return features



#### Processing full data

In [230]:
full_data = CreditReport(json_path = 'credit_report.json', ref_date = datetime.datetime(2021,1,1)).all_features()
full_data.head()

,application_id,rating_other_bad,rating_other_good,rating_retail_bad,rating_retail_good,rating_telecom_bad,rating_autoloan_bad,rating_autoloan_good,rating_homeloan_bad,rating_telecom_good,...,payment_hist_sum_duration,payment_hist_nb_open,payment_hist_nb_overdraft,is_male,age,dependants,nationality,has_id,nb_updates,time_to_update
0,9711360,0,3,0,2,0,0,0,0,0,...,7103,3,2,1,29,0,Nigeria,0,0.0,-1.0
1,9714953,0,3,0,12,0,0,0,0,0,...,2072,4,0,0,35,0,Nigeria,1,2.0,11.0
2,9714978,0,1,1,1,0,0,0,0,0,...,375,3,0,0,37,0,Nigeria,0,0.0,-1.0


#### Processing data for a list of IDs

In [231]:
ids_data = CreditReport(json_path = 'credit_report.json', ids = [9711360, 9714953],  ref_date = datetime.datetime(2021,1,1)).all_features()
ids_data.head()

,application_id,rating_other_bad,rating_other_good,rating_retail_bad,rating_retail_good,rating_telecom_bad,rating_autoloan_bad,rating_autoloan_good,rating_homeloan_bad,rating_telecom_good,...,payment_hist_sum_duration,payment_hist_nb_open,payment_hist_nb_overdraft,is_male,age,dependants,nationality,has_id,nb_updates,time_to_update
0,9711360,0,3,0,2,0,0,0,0,0,...,7103,3,2,1,29,0,Nigeria,0,0.0,-1.0
1,9714953,0,3,0,12,0,0,0,0,0,...,2072,4,0,0,35,0,Nigeria,1,2.0,11.0


#### Processing data for a single ID

In [232]:
id_data = CreditReport(json_path = 'credit_report.json', ids = [9711360],  ref_date = datetime.datetime(2021,1,1)).all_features()
id_data.head()

,application_id,rating_other_bad,rating_other_good,rating_retail_bad,rating_retail_good,rating_telecom_bad,rating_autoloan_bad,rating_autoloan_good,rating_homeloan_bad,rating_telecom_good,...,payment_hist_sum_duration,payment_hist_nb_open,payment_hist_nb_overdraft,is_male,age,dependants,nationality,has_id,nb_updates,time_to_update
0,9711360,0,3,0,2,0,0,0,0,0,...,7103,3,2,1,29,0,Nigeria,0,0,-1


#### Processing a single category of features

In [235]:
full_data = CreditReport(json_path = 'credit_report.json', ids = [9711360],  ref_date = datetime.datetime(2021,1,1))
full_data.specific_features(full_data.payment_history)

,application_id,payment_hist_avg_installment,payment_hist_max_installment,payment_hist_avg_overdue,payment_hist_max_overdue,payment_hist_avg_duration,payment_hist_max_duration,payment_hist_sum_duration,payment_hist_nb_open,payment_hist_nb_overdraft
0,9711360,19351,24368,3434,22441,1775,5843,7103,3,2
